In [ ]:
!pip install -q tensorflow==2.14.0 streamlit seaborn scikit-learn

import os, json, shutil
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print('TensorFlow', tf.__version__)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
BASE_DIR = '/content/drive/MyDrive/fish_classification'  # change if you want
DATA_DIR = os.path.join(BASE_DIR, 'data')
MODEL_DIR = os.path.join(BASE_DIR, 'saved_models')
RESULTS_DIR = os.path.join(BASE_DIR, 'results')
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)


In [ ]:
# TODO: Upload or unzip dataset into DATA_DIR/train, DATA_DIR/val, DATA_DIR/test

In [ ]:
IMG_SIZE = (224,224)
BATCH_SIZE = 32
EPOCHS = 20
LR = 1e-4
SEED = 42


In [ ]:
train_dir = os.path.join(DATA_DIR, 'train')
val_dir = os.path.join(DATA_DIR, 'val')
test_dir = os.path.join(DATA_DIR, 'test')

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=25,
                                   width_shift_range=0.12,
                                   height_shift_range=0.12,
                                   shear_range=0.12,
                                   zoom_range=0.12,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', seed=SEED)
val_gen = test_datagen.flow_from_directory(val_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False, seed=SEED)
test_gen = test_datagen.flow_from_directory(test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode='categorical', shuffle=False, seed=SEED)


In [ ]:
base = keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
base.trainable = False
inputs = keras.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.25)(x)
outputs = layers.Dense(len(train_gen.class_indices), activation='softmax')(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(LR), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


In [ ]:
ckpt = os.path.join(MODEL_DIR, 'mobilenetv2_best.h5')
callbacks = [
    keras.callbacks.ModelCheckpoint(ckpt, monitor='val_accuracy', save_best_only=True),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
]

history = model.fit(train_gen, validation_data=val_gen, epochs=EPOCHS, callbacks=callbacks)


In [ ]:
with open(os.path.join(MODEL_DIR, 'class_indices.json'), 'w') as f:
    json.dump(train_gen.class_indices, f)

plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.legend()
plt.title('Accuracy')
plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.legend()
plt.title('Loss')
plt.show()


In [ ]:
best = keras.models.load_model(ckpt)
steps = int(np.ceil(test_gen.samples / test_gen.batch_size))
preds = best.predict(test_gen, steps=steps)
y_pred = preds.argmax(axis=1)
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(test_gen.classes, y_pred, target_names=list(train_gen.class_indices.keys())))


In [ ]:
app_code = """
# Streamlit app for Multiclass Fish Image Classification
import streamlit as st
from tensorflow import keras
from PIL import Image
import numpy as np
import json
import os

MODEL_PATH = 'saved_models/mobilenetv2_best.h5'

@st.cache(allow_output_mutation=True)
def load_model(path):
    return keras.models.load_model(path)

if os.path.exists(MODEL_PATH):
    model = load_model(MODEL_PATH)
    with open('saved_models/class_indices.json','r') as f:
        class_map = json.load(f)
    inv_map = {int(v):k for k,v in class_map.items()}
else:
    st.stop()

st.title('🐟 Fish Species Classifier')
uploaded = st.file_uploader('Upload an image', type=['jpg','jpeg','png'])
if uploaded:
    img = Image.open(uploaded).convert('RGB')
    st.image(img,use_column_width=True)
    img = img.resize((224,224))
    arr = np.array(img)/255.0
    preds = model.predict(np.expand_dims(arr,0))[0]
    idx = preds.argmax()
    st.success(f'Prediction: {inv_map[idx]} — {preds[idx]:.2f}')
    st.write({inv_map[i]: float(p) for i,p in enumerate(preds)})
"""

with open(os.path.join(BASE_DIR,'app_streamlit.py'),'w') as f:
    f.write(app_code)

print('Streamlit app saved to', os.path.join(BASE_DIR,'app_streamlit.py'))
